### What you will find here
- Loading a single document
- Using the primary prompt templates used in llama-index
    + Load an LLM 
    + `chat` method that takes in a list of messages, to simulate a chat session. 
    + `Prompt` method that allows you do QA. 
- Quick check on limitation about this approach

In [12]:
import os
from llama_index.llms.openai import OpenAI
from llama_index.core import Prompt
from llama_index.core.llms import ChatMessage

### Load Single Document

In [4]:
file = os.path.join(
    os.environ["DATA_DIR"],
    "williams_family/biographies/Viviana Williams_bio.txt"
)
with open(file, "r") as f:
    context = f.read()

### Load an LLM

In [5]:
llm = OpenAI(model="gpt-3.5-turbo", temperature=0)

### Chat Example

In [6]:
CHAT_SYSTEM_CONTENT = """
    Given the context information and not prior knowledge,
    answer to the question, as briefly as possible.
    Structure your response as a list of facts.
"""

Asking questions about the family member that is covered in the loaded document.

In [10]:
chat_history = [
    ChatMessage(role="system", content=CHAT_SYSTEM_CONTENT),
    ChatMessage(role="assistant", content=context),
    ChatMessage(role="user", content=f"When did Viviana Williams born?"),
]
response = llm.chat(chat_history)
print(response.message)

assistant: Viviana Williams was born on December 11, 1963.


Asking question about a a family member not covered in the document.
We can quickly see the limitations of loading a single document!

Also, what if there is information about the same family member across different documents?

In [11]:
chat_history = [
    ChatMessage(role="system", content=CHAT_SYSTEM_CONTENT),
    ChatMessage(role="assistant", content=context),
    ChatMessage(role="user", content=f"When did Sora Williams born?"),
]

response = llm.chat(chat_history)
print(response.message)

assistant: Sora Williams was not mentioned in the provided context information.


### Prompt

In [13]:
text_qa_template = Prompt(
    """
    Context information is below.
    ---------------------
    {context_str}"
    ---------------------
    Given the context information and not prior knowledge,
    answer the question: {query_str}, as briefly as possible.
    Structure your response as a list of facts.
    """
)

refine_template = Prompt(
    """
    We have the opportunity to refine the original answer
    (only if needed) with some more context below.
    ------------
    {context_msg}
    ------------
    Given the new context, refine the original answer to better
    answer the question: {query_str}, as briefly as possible.
    Structure your response as a list of facts.
    If the context isn't useful, output the original answer again.
    Original Answer: {existing_answer}
    """
)

In [14]:
question = f"When was Viviana Williams born?"
prompt = text_qa_template.format(context_str=context, query_str=question)
response = llm.complete(prompt)
print(response.text)

- Viviana Williams was born on December 11, 1963.


In [15]:
question = "Give more information. For example, tell me the name of their parents and children."
prompt = refine_template.format(context_msg=context, query_str=question, existing_answer=response)
refined_response = llm.complete(prompt)
print(refined_response.text)

Refined Answer:
- Viviana Williams was born on December 11, 1963 to Immanuel Williams and Tessa Johnson.
- Viviana married Lalit Wilson and they had a son named David in 1988.
- Viviana passed away unexpectedly in 2010, leaving behind her husband Lalit and son David.
